In [ ]:
from skimage.io import imread as skimread
from pathlib import Path
import torch
import numpy as np
from tnia.deeplearning.dl_helper import quantile_normalization
from torchvision import transforms
from torchvision.transforms import v2

In [ ]:
from pathlib import Path
import os

tnia_images_path = Path(r"D:\images")
parent_path = r'C:\Users\bnort\Documents\bcs-stardist\images\Columbia_Semantic_Sparse'
model_path = os.path.join(parent_path,'models')

In [ ]:
net = torch.load( Path(model_path) / 'model1')
device = torch.device("cuda")
net.to(device)

In [ ]:
from skimage.io import imread
im = imread(r'C:\Users\bnort\Documents\bcs-stardist\images\Columbia_Semantic_Sparse\20240910205928_SEP10ERSP2424346.jpg')
im = np.transpose(im, (2,0,1))  
im.shape
im = im[:,:2048,:2048]

In [ ]:
# Visualize predictions
from tnia.deeplearning.dl_helper import quantile_normalization

#im, raw = train_dataset[10]

net.eval()

print(im.shape, im.min(), im.max())
im = quantile_normalization(im)
im = im.astype(np.float32)
print(im.shape, im.min(), im.max())
im_tensor = torch.from_numpy(im).unsqueeze(0).to(device)       
print(im_tensor.shape)

logits=net(im_tensor)


In [ ]:
from tnia.plotting.plt_helper import imshow_multi2d

im = np.transpose(im, (1,2,0))

logits.shape
c1 = logits[0,0,:,:].cpu().detach().numpy()
c2 = logits[0,1,:,:].cpu().detach().numpy()
c3 = logits[0,2,:,:].cpu().detach().numpy()

fig = imshow_multi2d([im, c1, c2, c3], ['Image', 'Class 1', 'Class 2', 'Class 3'], 1, 4, 10, 10,colormaps=['gray', 'viridis', 'viridis', 'viridis'])

In [ ]:
# go through 3 classes and take highest

class1 = (c1>c2)&(c1>c3)
class2 = (c2>c1)&(c2>c3)
class3 = (c3>c1)&(c3>c2)

labels = class1+2*class2+3*class3

In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_image(im)
viewer.add_labels(labels)
#viewer.add_image(c1)
#viewer.add_image(c2)
#viewer.add_image(c3)

In [ ]:
thresholded = c2 > 3.0
viewer.add_labels(thresholded*2)

In [ ]:
thresholded3 = c3 > 10.0
viewer.add_labels(thresholded3*3)